In [1]:
import joblib
import pickle
import minisom
import laytr

First, we need to create a som

In [2]:
kfeats = joblib.load("../repo_utils/kfeat.jl")
som = minisom.MiniSom(15, 15, 64, sigma=1.5, learning_rate=1,
                      topology='hexagonal',
                      neighborhood_function='gaussian', 
                      activation_distance='euclidean',
                      random_seed=2023
                     )
som.train_batch(kfeats, 10_000, verbose=True)
with open('chr22_example.som', 'wb') as outfile:
    pickle.dump(som, outfile)

 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 0.10895308115856686


Next, we map our data to the SOM. We can do this via the command line with `laytr map` or programmatically

In [4]:
m_map = laytr.map_to_som(kfeats, som)